In [26]:
import pandas as pd
#from tensorflow import keras
import keras
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import statsmodels.api as sm
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import metrics
import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow.compat'

In [ ]:
# prepare training and test data
def prepare_data():
    df = pd.read_excel('1128 新标注数据 吸气18_19 呼气1_12随机8个位置.xlsx')
    df['I/E']=df['I/E'].apply(lambda x:2 if x==-1 else x)
    y = df['I/E'].astype('int64')
    x = df.iloc[:, 0:25]
    x_t = x.T
    scaler_x = StandardScaler()
    scaler_x.fit(x_t)
    npx = scaler_x.transform(x_t)
    npx=npx.T
    y = np.array(y)
    len(y)
    npx = npx.reshape(39573,25,1)
    x_train, x_test, y_train, y_test = train_test_split(npx,y,test_size=0.3,random_state=12,shuffle=True)
    return x_train, x_test, y_train, y_test

In [ ]:
# build a lstm model
def lstm_base_model():
    input_data = keras.layers.Input(shape=(25,1))
    lstm_out =keras.layers.LSTM(8,activation='tanh',return_sequences=True)(input_data)
    lstm_out =keras.layers.Dropout(0.2)(lstm_out)
    # lstm_out =keras.layers.LSTM(8,activation='tanh',return_sequences=True)(lstm_out)
    # lstm_out =keras.layers.Dropout(0.2)(lstm_out)
    lstm_out =keras.layers.LSTM(8,activation='tanh')(lstm_out)
    lstm_out =keras.layers.Dropout(0.2)(lstm_out)
    dense_out = keras.layers.Dense(4,activation='softmax')(lstm_out)
    model = keras.Model(input_data, dense_out)
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return model

# sparse_categorical_crossentropy

In [ ]:
#plot Loss and Accuracy 
def plot_history(history):
    history.history.keys()
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    plt.figure(figsize=(6, 6), dpi=400)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy', color="#d6c5e8")
    plt.ylabel('Accuracy', fontsize=14)
    plt.xlabel('Epoches', fontsize=14)
    plt.legend(fontsize=10)
    plt.savefig("accuracy1.png", dpi=400)
    plt.figure(figsize=(5,5), dpi=400)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss', color="#d6c5e8")
    plt.ylabel('Loss',fontsize=14)
    plt.xlabel('Epoches',fontsize=14)
    plt.legend(loc='upper right',fontsize=10, ncol=1, frameon=True)
    plt.savefig("val_loss1", dpi=400)

In [ ]:
def paint_confusion_matrix(x_test, y_test):
    sns.set()
    score = model.evaluate(x_test, y_test, verbose=0)
    pre = model.predict(x_test).argmax(1)
    C2 = metrics.confusion_matrix(y_test, pre, labels=[0,1,2])
    plt.figure()
    classes = [0,1,2]
    plt.imshow(C2, interpolation='nearest', cmap=plt.cm.Oranges)  # 按照像素显示出矩阵
    plt.title('confusion_matrix')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)
     
    thresh = C2.max() / 2.
    # iters = [[i,j] for i in range(len(classes)) for j in range((classes))]
    # ij配对，遍历矩阵迭代器
    iters = np.reshape([[[i, j] for j in range(len(classes))] for i in range(len(classes))], (C2.size, 2))
    for i, j in iters:
        plt.text(j, i, format(C2[i, j]))  # 显示对应的数字
     
    plt.ylabel('Real label')
    plt.xlabel('Prediction')
    plt.tight_layout()
    plt.show()
    # 误触发率计算
    sum = 0
    for i in range(len(classes)):
        for j in range(len(classes)):
            sum += C2[i,j]
    sum_false = 0
    for i in range(len(classes)):
        for j in range(len(classes)):
            if i != j:
                sum_false += C2[i,j]
    falsetriggerrate = sum_false/sum     # 误触发率
    sns.set()
    C2=np.array([[3057,15,16],[35,1142,9],[37,8,1144]])
    plt.figure()
    classes = [0,1,2]
    plt.imshow(C2, interpolation='nearest', cmap=plt.cm.viridis)  # 按照像素显示出矩阵
    plt.title('confusion_matrix')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, ["Non-I/E","Expiratory","Inspiratory"])
    plt.yticks(tick_marks, ["Non-I/E","Expiratory","Inspiratory"], rotation=90)
     
    thresh = C2.max() / 2.
    iters = np.reshape([[[i, j] for j in range(len(classes))] for i in range(len(classes))], (C2.size, 2))
    for i, j in iters:
        if i == j:
            plt.text(j, i, format(C2[i, j]),ha='center', va='center', color='white')  # 显示对应的数字
        else:
            plt.text(j, i, format(C2[i, j]),ha='center', va='center', color='black')  # 显示对应的数字
    plt.grid(False)
    plt.ylabel('Real label')
    plt.xlabel('Prediction')
    plt.tight_layout()
    plt.savefig("confusion_matrix.png",dpi=300)

In [ ]:
# 1. prepare data
x_train, x_test, y_train, y_test = prepare_data()
# 2. build structure of lstm
model = lstm_base_model()
model.summary()
# 3 train model
history_training = model.fit(x_train,y_train,validation_split=0.1,epochs=600, batch_size=16)
# 4 observe result
plot_history(history_training)
# 5. predict result by test data
paint_confusion_matrix(x_test, y_test)